In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
filter_tazs, filter_mazs = vh.get_filters(zone_set,
                                          affected_tazs=affected_tazs,
                                          affected_mazs=affected_mazs
                                        )

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']


In [ ]:
name_dict = {'escort': "Escorting", 
             'work': "Work", 
             'school': "School", 
             'othmaint': "Other-Maintenance",
             'othdiscr': "Other-Discretionary", 
             'shopping': "Shopping",
             'atwork': "At-work Subtour", 
             'eatout': "Eat Out", 
             'social' : "Social", 
             'univ': "University"}

## Joint Tour Participation

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## Joint Tour Destinations

In [ ]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','number_of_participants','destination','primary_purpose'])

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','number_of_participants','destination','primary_purpose'])


In [ ]:
print(f"Filtering tours by destination. Original tours: {len(base_tour)}")
base_tour = base_tour[filter_tazs(base_tour.destination)]
print(f"Tours after filtering: {len(base_tour)}")

In [ ]:
df = base_tour[base_tour.number_of_participants > 1].merge(build_tour,
                                                           how='left',
                                                           left_index=True,
                                                           right_index=True,
                                                           suffixes=('_base','_build'))

df = (df.destination_base == df.destination_build).value_counts()
df.index = df.index.map({True:'No Change',False:'Change'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame(),["count"])
fig.show()

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly


for key, value in name_dict.items():

    if len(base_tour[
        (base_tour.number_of_participants > 1)
        & (base_tour.primary_purpose == key)
        ]) == 0:
        continue
    
    print(f"#### {value}")
    df = base_tour[
        (base_tour.number_of_participants > 1)
        & (base_tour.primary_purpose == key)
        ].merge(build_tour,
                how='left',
                left_index=True,
                right_index=True,
                suffixes=('_base','_build'))
    
    df = (df.destination_base == df.destination_build).value_counts()
    df.index = df.index.map({True:'No Change',False:'Change'})
    print(str(df).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df.to_frame(),["count"])
    
    fig.show()
    display(Markdown(" "))

## Joint Tour Scheduling

In [ ]:
print("Lorem ipsum") # Placeholder for WSP